# VacationPy
----

In this notebook, we will use a table of weather data to show current humidity, find some ideal vacation spots, and a nearby hotel (money is no object):
* A pleasant temperature, between 70 and 80 degrees Farenheit
* Low winds, less than 10 mph
* An azure sky on a cloudless day

### Import libraries and API keys

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import gmaps

# Import API key
from api_keys import g_key

### Load the weather data into a DataFrame

In [2]:
city_weather = pd.read_csv("../WeatherPy/output_data/cities.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '900px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Create the fiture
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(city_weather[["Lat", "Lng"]], weights=city_weather["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

### Create a DataFrame of tempting vacation cities, that fit the weather criteria

In [4]:
# Filter and Copy the DataFrame
vacation_cities = city_weather[
    (city_weather["Max_Temp"]>70.0) \
    & (city_weather["Max_Temp"]<80.0) \
    & (city_weather["Wind_Speed"]<10.0) \
    & (city_weather["Cloudiness"]==0)].copy()

### Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
* The nearbysearch API function did not allow me to set a radius of 5000 meters AND rank by distance.
* I chose to use the findplacefromtext API function, wherein I could request "closest" (imply a ranking) AND a search radius.

In [5]:
# Initialize an empty list of nearby hotels, and the base URL for the Google Places API
hotel_name_list = list()
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

# Loop thru the vacation cities, and find the closest lodging
for index, row in vacation_cities.iterrows():
    params = {
        "key" : g_key,
        "input" : "closest lodging",
        "inputtype" : "textquery",
        "language" : "en",
        "fields" : "formatted_address,name",
        "locationbias" : f"circle:5000@{row['Lat']},{row['Lng']}"
    }
    json = requests.get(base_url, params=params).json()
    # When the API fails to find a hotel that suits the parameterized criteria, it simply returns a hotel near me in Texas.
    if "TX" in json['candidates'][0]['formatted_address']:
        hotel_name = "No hotel within 5km"
    else:
        hotel_name = json['candidates'][0]['name']
    hotel_name_list.append(hotel_name)
    
# Merge the list of hotel names into the DataFrame
vacation_cities['Hotel'] = hotel_name_list

### Create Markers with Information Windows for Vacation Cities and Nearby Hotels

In [6]:
# Use this template for the information windows
info_box_template = """
<dl>
<dt>Nearby Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# For clarity, initialize lists of (1) information window content and (2) locations
hotel_info = [info_box_template.format(**row) for index, row in vacation_cities[["Hotel","City","Country"]].iterrows()]
locations = vacation_cities[["Lat", "Lng"]]

### Display the Map
* Humidity from the weather table is indicated by a heat map.
* Ideal vacation sites are marked.

In [7]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display the map
fig

Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…